In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime


In [6]:
from xbbg import blp


## Downloading Data for 3 Maturities

In [11]:
def download_option_data(tickers, expiration_dates):
    
    # empty array to store all the downloaded options
    all_options = []

    # get today's date for time to maturity calculation
    today = datetime.today()
    # getting the interest rate through the closing price of 13 Week Treasury Bill
    fed_funds = yf.Ticker("^IRX").history(period="1d")['Close'].iloc[-1]  

    # iterating through the list of provided tickers
    for ticker in tickers:
        try:
            # fetch most recent 15-minute interval closing price fro spot price
            spot_price = yf.Ticker(ticker).history(period="1d", interval="5m")["Close"].iloc[-1]
        except Exception as e:
            # error handling for the spot price fetching
            print(f"Error fetching 5-min close price for {ticker}: {e}")
            spot_price = None  # Assign None if price fetch fails

        # for each expirations dates getting the option data
        for exp_date in expiration_dates:
            try:
                # convert expiration date to datetime datatype
                exp_datetime = datetime.strptime(exp_date, "%Y-%m-%d")
                time_to_maturity = (exp_datetime - today).days / 365.0  # Convert to years

                # option chain from yfinance
                option_chain = yf.Ticker(ticker).option_chain(exp_date)
                # sperating the calls and puts chains
                calls = option_chain.calls
                puts = option_chain.puts

                # add identifying columns
                calls["ticker"] = ticker
                calls["expiration"] = exp_date
                calls["optionType"] = "Call"
                calls["timeToMaturity"] = round(time_to_maturity, 6)
                calls["spotPrice"] = round(spot_price,6)

                puts["ticker"] = ticker
                puts["expiration"] = exp_date
                puts["optionType"] = "Put"
                puts["timeToMaturity"] = round(time_to_maturity, 6)
                puts["spotPrice"] = round(spot_price,6)

                # combine calls & puts
                all_options.append(calls)
                all_options.append(puts)

            except Exception as e:
                print(f"Error fetching options for {ticker} ({exp_date}): {e}")

    # combine all data into a single DataFrame
    if not all_options:
        return None

    options_data = pd.concat(all_options, ignore_index=True)
    # adding interest rate to all the option data
    options_data["interestRate"] = round(fed_funds, 6)

    # rearrange columns and only getting the necessary fields
    desired_columns = ["ticker", "expiration", "timeToMaturity", "strike", "optionType",
                       "lastPrice", "bid", "ask", "impliedVolatility", "inTheMoney", "spotPrice", "interestRate"]
    options_data = options_data[desired_columns]

    return options_data

In [12]:
# get option data for NVDA fro three different maturities
tickers = ["NVDA"]
expiration_dates = ["2025-03-21", "2025-04-17", "2025-05-16"]
options_data = download_option_data(tickers, expiration_dates)

In [13]:
options_data.head()

,ticker,expiration,timeToMaturity,strike,optionType,lastPrice,bid,ask,impliedVolatility,inTheMoney,spotPrice,interestRate
0,NVDA,2025-03-21,0.076712,5.0,Call,134.70,134.35,135.10,6.062502,True,139.360596,4.205
1,NVDA,2025-03-21,0.076712,10.0,Call,124.00,129.35,130.15,4.671879,True,139.360596,4.205
2,NVDA,2025-03-21,0.076712,15.0,Call,104.65,124.30,125.05,3.769532,True,139.360596,4.205
3,NVDA,2025-03-21,0.076712,20.0,Call,120.03,119.35,120.10,3.339845,True,139.360596,4.205
4,NVDA,2025-03-21,0.076712,25.0,Call,106.22,114.35,115.10,2.953128,True,139.360596,4.205


In [14]:
options_data.to_csv("NVDA_options_data.csv")

In [8]:
import requests

session = requests.Session()

In [10]:
def download_option_data_xbbg(ticker):
    """Fetches option data for the first 3 expiration dates using xbbg (Bloomberg)."""

    # Step 1: Retrieve the first 3 expiration dates
    expiration_dates = blp.bdp(f"{ticker} Equity", "OPT_EXPIRE_DT_LIST")["OPT_EXPIRE_DT_LIST"]
    
    if expiration_dates is None or len(expiration_dates) == 0:
        print(f"No expiration dates found for {ticker}.")
        return None

    expiration_dates = sorted(expiration_dates)[:3]  # Select first 3 expirations

    # Step 2: Fetch spot price
    spot_price = blp.bdp(f"{ticker} Equity", "PX_LAST")["PX_LAST"].iloc[0]

    # Step 3: Fetch options for each expiration date
    all_options = []
    today = datetime.today()

    for exp_date in expiration_dates:
        exp_str = exp_date.strftime("%Y-%m-%d")  # Convert to string format
        time_to_maturity = (exp_date - today).days / 365.0  # Convert to years

        # Fetch all options for this expiration
        option_chain = blp.bds(f"{ticker} Equity", "OPT_CHAIN")
        
        if option_chain is None or option_chain.empty:
            print(f"No option data for {ticker} ({exp_str})")
            continue

        # Process calls and puts
        for idx, row in option_chain.iterrows():
            option_type = "Call" if row["Put Call"] == "Call" else "Put"
            option_data = {
                "ticker": ticker,
                "expiration": exp_str,
                "timeToMaturity": round(time_to_maturity, 6),
                "strike": row["Strike"],
                "optionType": option_type,
                "lastPrice": row["Last Price"],
                "bid": row["Bid"],
                "ask": row["Ask"],
                "impliedVolatility": row["Implied Volatility"],
                "inTheMoney": row["In The Money"],
                "spotPrice": round(spot_price, 6),
            }
            all_options.append(option_data)

    # Combine all data into a DataFrame
    if not all_options:
        return None

    options_data = pd.DataFrame(all_options)

    # Select relevant columns
    desired_columns = ["ticker", "expiration", "timeToMaturity", "strike", "optionType",
                       "lastPrice", "bid", "ask", "impliedVolatility", "inTheMoney", "spotPrice"]
    options_data = options_data[desired_columns]

    return options_data

In [ ]:


ticker = "NVDA"
option_data = download_option_data_xbbg(ticker)
print(option_data)


## Visualize Results

In [3]:
df = pd.read_csv('NVDA_options_data.csv')
NVDA_spot = df.iloc[1, -2]

NVDA_spot

139.360596

In [4]:
NVDA_output_df = pd.read_csv('output_NVDA_test.csv')
NVDA_output_df.head()

,Ticker,Expiration,TimeToMaturity,Strike,OptionType,LastPrice,Bid,Ask,ImpliedVolatility,BisectionIV,BisectionTime,Bs_price,Binom_price,American_binom_price,InTheMoney
0,NVDA,2025-03-21,0.076712,5,Call,134.70,134.35,135.10,6.06250,0.00010,0.010880,134.377,134.377,134.377,True
1,NVDA,2025-03-21,0.076712,10,Call,124.00,129.35,130.15,4.67188,0.00010,0.000603,129.393,129.393,129.393,True
2,NVDA,2025-03-21,0.076712,15,Call,104.65,124.30,125.05,3.76953,0.00010,0.000674,124.409,124.409,124.409,True
3,NVDA,2025-03-21,0.076712,20,Call,120.03,119.35,120.10,3.33985,0.00010,0.000379,119.425,119.425,119.425,True
4,NVDA,2025-03-21,0.076712,25,Call,106.22,114.35,115.10,2.95313,2.94676,0.002919,114.725,114.730,114.730,True


In [43]:
filtered_df = NVDA_output_df[
    (NVDA_spot / NVDA_output_df['Strike'] >= 0.9) &
    (NVDA_spot / NVDA_output_df['Strike'] <= 1.1) &
    (NVDA_output_df['Strike']%5 == 0)
]

In [44]:
filtered_df = filtered_df[['Expiration','Strike', 'OptionType', 'Ask', 'Bid', 'Bs_price', 'Binom_price', 'American_binom_price']]


In [45]:
filtered_df

,Expiration,Strike,OptionType,Ask,Bid,Bs_price,Binom_price,American_binom_price
78,2025-03-21,130,Call,15.05,14.95,15.00000,15.00150,15.00150
81,2025-03-21,135,Call,11.90,11.80,11.85000,11.84840,11.84840
86,2025-03-21,140,Call,9.20,9.15,9.17498,9.17722,9.17722
91,2025-03-21,145,Call,7.00,6.90,6.95000,6.95048,6.95048
96,2025-03-21,150,Call,5.15,5.05,5.10000,5.10032,5.10032
401,2025-03-21,130,Put,4.80,4.75,4.77500,4.77639,4.78854
404,2025-03-21,135,Put,6.65,6.60,6.62500,6.62454,6.64347
409,2025-03-21,140,Put,8.95,8.90,8.92500,8.92717,8.95577
414,2025-03-21,145,Put,11.75,11.65,11.70000,11.69780,11.74010
419,2025-03-21,150,Put,14.95,14.85,14.90000,14.90180,14.96180
